In [66]:
from sqlalchemy import create_engine, inspect
engine = create_engine("sqlite+pysqlite:////workspaces/D2I-Jupyter-Notebook-Tools/SQL_training/chinook.db", echo=True)

connection = engine.connect()
inspection = inspect(engine)
inspection.get_table_names()


2023-09-21 13:40:40,286 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:40:40,287 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-09-21 13:40:40,288 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-21 13:40:40,290 INFO sqlalchemy.engine.Engine ROLLBACK


['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'playlist_track',
 'playlists',
 'tracks']

In [67]:
from sqlalchemy import MetaData, Table

metadata = MetaData()

artists = Table('artists', metadata, autoload_with=engine)

# print(track.name)

# print(tracks.c.keys())

print(repr(artists)) # table definition


2023-09-21 13:40:40,333 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:40:40,335 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("artists")
2023-09-21 13:40:40,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-21 13:40:40,338 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-09-21 13:40:40,338 INFO sqlalchemy.engine.Engine [raw sql] ('artists',)
2023-09-21 13:40:40,340 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("artists")
2023-09-21 13:40:40,340 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-21 13:40:40,341 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("artists")
2023-09-21 13:40:40,342 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-21 13:40:40,343 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 

In [68]:
# from sqlalchemy import text

# with connection as con:
#     stmt = "SELECT * FROM tracks"
#     results_proxy = con.execute(text(stmt))
#     results = results_proxy.fetchall()

# first_row = results[0]

# # print(first_row) # print entire first row

# # print(first_row.Name) # print Name column from first row

# # unpack multiple rows - column Name
# multiple_rows = results[:10]
# for row in multiple_rows:
#     print(row.Name)


In [69]:
# with connection as con:
#     stmt = "SELECT * FROM artists"
#     results_proxy = con.execute(text(stmt))
#     results = results_proxy.fetchall()

# first_row = results[0]

# print(first_row) # print entire first row

In [71]:
from sqlalchemy import select

with engine.connect() as con:
    stmt = select(artists)
    results_proxy = con.execute(stmt)
    results = results_proxy.fetchmany(size=10)
print(results)

2023-09-21 13:40:47,841 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:40:47,842 INFO sqlalchemy.engine.Engine SELECT artists."ArtistId", artists."Name" 
FROM artists
2023-09-21 13:40:47,843 INFO sqlalchemy.engine.Engine [cached since 7.388s ago] ()
2023-09-21 13:40:47,844 INFO sqlalchemy.engine.Engine ROLLBACK
[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]


In [76]:
from sqlalchemy import or_

with engine.connect() as con:
    stmt = select(artists)
    stmt = stmt.where(or_(artists.columns.ArtistId == 1,
                          artists.columns.Name == 'Alice In Chains' ))
    results = con.execute(stmt).fetchall()
print(results)

2023-09-21 13:49:27,293 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:49:27,294 INFO sqlalchemy.engine.Engine SELECT artists."ArtistId", artists."Name" 
FROM artists 
WHERE artists."ArtistId" = ? OR artists."Name" = ?
2023-09-21 13:49:27,295 INFO sqlalchemy.engine.Engine [cached since 10.94s ago] (1, 'Alice In Chains')
2023-09-21 13:49:27,297 INFO sqlalchemy.engine.Engine ROLLBACK
[(1, 'AC/DC'), (5, 'Alice In Chains')]


In [81]:
with engine.connect() as con:
    stmt = select(artists).order_by(artists.columns.Name.desc())
    results = con.execute(stmt).fetchall()
print(results)

2023-09-21 13:54:48,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:54:48,220 INFO sqlalchemy.engine.Engine SELECT artists."ArtistId", artists."Name" 
FROM artists ORDER BY artists."Name" DESC
2023-09-21 13:54:48,221 INFO sqlalchemy.engine.Engine [cached since 92.82s ago] ()
2023-09-21 13:54:48,223 INFO sqlalchemy.engine.Engine ROLLBACK
[(155, 'Zeca Pagodinho'), (168, "Youssou N'Dour"), (212, 'Yo-Yo Ma'), (255, 'Yehudi Menuhin'), (181, 'Xis'), (211, 'Wilhelm Kempff'), (154, 'Whitesnake'), (73, 'Vinícius E Qurteto Em Cy'), (74, 'Vinícius E Odette Lara'), (71, 'Vinícius De Moraes & Baden Powell'), (72, 'Vinícius De Moraes'), (75, 'Vinicius, Toquinho & Quarteto Em Cy'), (153, 'Velvet Revolver'), (21, 'Various Artists'), (152, 'Van Halen'), (151, 'UB40'), (150, 'U2'), (70, 'Toquinho & Vinícius'), (231, 'Ton Koopman'), (146, 'Titãs'), (145, 'Tim Maia'), (144, 'The Who'), (143, 'The Tea Party'), (142, 'The Rolling Stones'), (174, 'The Postal Service'), (200, 'The Posies'), (

In [83]:
from sqlalchemy import func

with engine.connect() as con:
    stmt = select(func.sum(artists.columns.ArtistId))
    results = con.execute(stmt).scalar()
print(results)


2023-09-21 13:58:45,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-21 13:58:45,666 INFO sqlalchemy.engine.Engine SELECT sum(artists."ArtistId") AS sum_1 
FROM artists
2023-09-21 13:58:45,667 INFO sqlalchemy.engine.Engine [generated in 0.00253s] ()
2023-09-21 13:58:45,668 INFO sqlalchemy.engine.Engine ROLLBACK
37950
